### Importing all the necessary libraries

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout, Bidirectional
from keras.initializers import Constant
from keras.optimizers import Adam

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Parik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Parik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Parik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Read data

In [3]:
df_test = pd.read_csv('Self_testData.csv')
df_train = pd.read_csv('train.csv')

### Creating embeddings dictionary by loading Twitter GloVe (27B tokens) with 100-D vectors

In [4]:
import io

dictionary_embedding={}
with open('glove.twitter.27B.100d.txt','r', encoding="utf8") as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        dictionary_embedding[word]=vectors

### Using the "nlpaug" library, tweets can be augmented by replacing words in them with their synonyms.

In [5]:
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

aug = naw.SynonymAug(aug_src='wordnet')

def augmented_data (text, aug):
	augmented_text2 = aug.augment(text)
	augmented_text3 = aug.augment(text)

	return augmented_text2,augmented_text3,

for i in range(df_train.shape[0]):
	text = df_train["text"][i] 
	label = df_train["target"][i]

	id = int('0'+str(df_train["id"][i]))
	augmented1, augmented2 = augmented_data(text, aug)
	df1 = pd.DataFrame({"id":[id, id],"keyword":["",""],"location":["",""],"text":[augmented1, augmented2],"target":[label, label]})       
	df_train = df_train.append(df1,ignore_index = True)

print(df_train.shape[0])
df_train.to_csv("train_augmented.csv")

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

...

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignore_index = True)
C:\Users\Parik\AppData\Local\Temp\ipykernel_20516\784820058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df1,ignor

22839


### Building vocabulary from text

In [6]:
def build_vocab(texts):
    """
    :param sntcs: list of list of words
    :return: dictionary of words and their count
    """
    sntcs = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sntc in tqdm(sntcs):
        for word in sntc:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

### Examine the extent to which the embeddings cover the vocabulary.

In [7]:
import operator 

def check_coverage(vocab,embeddings_index):
    p = {}
    oov = {}
    q = 0
    i = 0
    for word in tqdm(vocab):
        try:
            p[word] = embeddings_index[word]
            q += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(p) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(q / (q + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [8]:
def build_vocab(texts):
    """
    :param texts: list of strings or a single string
    :return: dictionary of words and their count
    """
    if isinstance(texts, str):
        texts = [texts]
    sntcs = [sntc.split() for sntc in texts]
    vocab = {}
    for sntc in tqdm(sntcs):
        for word in sntc:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

### Performing text preprocessing on the train and test dataframes.

In [9]:
df = pd.concat([df_train,df_test])

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',str(text))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    pattern_of_emoji = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return pattern_of_emoji.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

def lower(text):
    words = text.split(" ")
    lower = " ".join([w.lower() for w in words])
    return lower

df['text']=df['text'].apply(lambda x : remove_URL(x))
df['text']=df['text'].apply(lambda x : remove_html(x))
df['text']=df['text'].apply(lambda x: remove_emoji(x))
df['text']=df['text'].apply(lambda x : remove_punct(x))
df['text']=df['text'].apply(lambda x : lower(x))

### Converting the data into a format suitable for input to the LSTM network. If a sntc exceeds 32 words, truncate it. Later, creating an embedding matrix.

In [10]:
MAX_LEN = 32
obj_tokenizer = Tokenizer()
obj_tokenizer.fit_on_texts(df['text'].values.tolist()) 
sequences = obj_tokenizer.texts_to_sequences(df['text'].values)
text_pad = pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
word_index = obj_tokenizer.word_index

words_count = len(word_index)+1
dim = 100

words_unknown = np.random.uniform(-1,1,size=dim).astype('float32')
words_unknown = words_unknown.reshape(1,dim)

embedding_matrix = np.zeros((words_count, dim))
for word, i in tqdm(word_index.items()):    
    if i > words_count:
        continue
    emb_vec=dictionary_embedding.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
    else:
        embedding_matrix[i]=words_unknown

embedding_matrix.shape

100%|████████████████████████████████████████████████████████████████████████| 33883/33883 [00:00<00:00, 563184.25it/s]


(33884, 100)

### Define model - Bidirectional LSTM layer

In [11]:
model=Sequential()

embedding = Embedding(words_count,dim,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(Bidirectional(LSTM(256, dropout=0.25, recurrent_dropout=0.2)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 100)           3388400   
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1024)              525312    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

#### - Specify callback functions for the LSTM model. 
#### - Use the ModelCheckpoint callback to save the best performing model during training.
#### - Using the EarlyStopping callback to stop training early if there is no improvement in the model's performance.

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
def get_callbacks():
    path_chckpt ='checkpoint_keras.h5'  
    dir_log='logs'   
    callback_chckpt = ModelCheckpoint(filepath=path_chckpt,
                                         monitor='val_accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         period=1)
    callbk_early_stopping = EarlyStopping(monitor='val_accuracy',
                                           patience=5,
                                           verbose=1)
    callbacks = [callback_chckpt, callbk_early_stopping]
    return callbacks

### Divide the data into training and validation (testing) sets, with a split ratio of 80:20. Train the model.

In [13]:
train = text_pad[:df_train.shape[0]]
test = text_pad[df_train.shape[0]:]

X_train,X_test,y_train,y_test=train_test_split(train,df_train['target'].values,test_size=0.2,random_state=40)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

history=model.fit(X_train,y_train,
                  batch_size=64,
                  epochs=25, # Main 25
#                   epochs=2,
                  validation_data=(X_test,y_test),
                  verbose=1,
                  callbacks = get_callbacks())

Shape of train (18271, 32)
Shape of Validation  (4568, 32)
Epoch 1/25
286/286 [==============================] - ETA: 0s - loss: 0.5317 - accuracy: 0.7561
Epoch 1: val_accuracy improved from -inf to 0.78919, saving model to checkpoint_keras.h5
286/286 [==============================] - 97s 317ms/step - loss: 0.5317 - accuracy: 0.7561 - val_loss: 0.4643 - val_accuracy: 0.7892
Epoch 2/25
286/286 [==============================] - ETA: 0s - loss: 0.4760 - accuracy: 0.7910
Epoch 2: val_accuracy improved from 0.78919 to 0.80823, saving model to checkpoint_keras.h5
286/286 [==============================] - 88s 307ms/step - loss: 0.4760 - accuracy: 0.7910 - val_loss: 0.4337 - val_accuracy: 0.8082
Epoch 3/25
286/286 [==============================] - ETA: 0s - loss: 0.4478 - accuracy: 0.8030
Epoch 3: val_accuracy improved from 0.80823 to 0.81502, saving model to checkpoint_keras.h5
286/286 [==============================] - 90s 317ms/step - loss: 0.4478 - accuracy: 0.8030 - val_loss: 0.4158 -

### Display the evaluation metrics for the validation data.

In [14]:
from sklearn import metrics

pred_y = model.predict(X_test)
pred_y = pred_y.round().astype('int')

print(metrics.accuracy_score(y_test,pred_y))
print(metrics.confusion_matrix(y_test,pred_y))
print(metrics.classification_report(y_test,pred_y))

143/143 [==============================] - 8s 50ms/step
0.8892294220665499
[[2359  255]
 [ 251 1703]]
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2614
           1       0.87      0.87      0.87      1954

    accuracy                           0.89      4568
   macro avg       0.89      0.89      0.89      4568
weighted avg       0.89      0.89      0.89      4568



### Using the trained model to make predictions on the labels of the tweets in the test data.

In [15]:
GloVe_train_pred = model.predict(test)
GloVe_train_pred_int = GloVe_train_pred.round().astype('int')

submission = pd.read_csv('sample_submission.csv')
submission['target'] = GloVe_train_pred_int
submission.to_csv("LSTM_Glove_non_augmented.csv", index=False, header=True)

102/102 [==============================] - 5s 47ms/step


### Calculating weighted probability scores for tweets by taking into account the "keyword" column, which is used to assign weights to distinguish between real and fake tweets.

In [16]:
keywords_unique = df_train.keyword.unique()
locations_unique = df_train.keyword.unique()

target_dict_keyword = {}
target_dict_location = {}

for key in keywords_unique:
    target_dict_keyword[key] = df_train[df_train['keyword']==key].target.values
for key in locations_unique:
    target_dict_location = df_train[df_train['keyword']==key].target.values

df_train['keyword'].fillna('empty',inplace=True)
answer = df_train[df_train['keyword']!='empty'].shape[0]
print(answer)

dict_keyword_prob = {}
dict_location_prob = {}
for key in target_dict_keyword.keys():
    count = 0
    for i in range(len(target_dict_keyword[key])):
        if(target_dict_keyword[key][i]==1):
            count += 1
    if(len(target_dict_keyword[key])!=0):
        dict_keyword_prob[key] = count/len(target_dict_keyword[key])

model_predictions = model.predict(test)
final_predictions = []
count = 0
df_test['keyword'].fillna('empty',inplace=True)
dict_keyword_prob['empty'] = 1
change_count_to_0 = 0
change_count_to_1 = 0

for i in range(len(model_predictions)):
    count += 1
    if(((dict_keyword_prob[df_test['keyword'].tolist()[i]])+model_predictions[i])/2 <= 0.5):
        if(model_predictions[i]>0.5):
            change_count_to_0 += 1
        final_predictions.append(0)
    else:
        if(model_predictions[i]<=0.5):
            change_count_to_1 += 1
        final_predictions.append(1)

submission['target'] = final_predictions
submission.to_csv("LSTM_Glove_aug_weighted.csv", index=False, header=True)

22778
102/102 [==============================] - 5s 46ms/step
